In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
  return series[..., np.newaxis].astype(np.float32)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [6]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [7]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0476 - last_time_step_mse: 0.0359 - val_loss: 0.0401 - val_last_time_step_mse: 0.0292
Epoch 2/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0349 - last_time_step_mse: 0.0227 - val_loss: 0.0311 - val_last_time_step_mse: 0.0181
Epoch 3/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0293 - last_time_step_mse: 0.0167 - val_loss: 0.0295 - val_last_time_step_mse: 0.0182
Epoch 4/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0276 - last_time_step_mse: 0.0158 - val_loss: 0.0267 - val_last_time_step_mse: 0.0147
Epoch 5/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0247 - last_time_step_mse: 0.0123 - val_loss: 0.0228 - val_last_time_step_mse: 0.0096
Epoch 6/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0221 - last_time_step_mse: 0.0090 - val_loss: 0.0226 - val_last_time_step_mse: 0.0091
Epoch 7/20
219/219 [==